### Convert hdf file to tif

In [68]:
def ListofExtensionAndName(directory,extension):
     
        if len(directory) != None:
            import os
            FilesList = []
            FileName=[]
            for root, subdirectory, files in os.walk(directory):
                for file in files:
                    if file.endswith(extension):
                        FilesList.append(os.path.join(root,file))
                        base=os.path.basename(file)
                        FileName.append(os.path.splitext(base)[0])

            return sorted(FilesList),sorted(FileName)
        else:
            print("no"+ extension +"file for this directory")

In [69]:
import time
import sys
import glob
from bs4 import BeautifulSoup
import os
import re
import requests
from osgeo import gdal,osr
import numpy

In [136]:
folder_list=glob.glob('../data/Modis_Fire_Date/2*/*/')
folder_list=sorted(folder_list)
len(folder_list)

228

In [138]:
folder_list=folder_list[168:]
len(folder_list)

60

In [140]:
folder_list[-1]

'../data/Modis_Fire_Date/2019/2019.12.01/'

In [141]:
for index,path in enumerate(folder_list,1):
    modis_status=False
    download_path=path
    
    # with this loop, we are checking the next month or next year file
    # if next file is started the downloading, this script is started    
    while modis_status is False:
        # last year of the product
        #################################
        if download_path.split('/')[-3]=='2019':
            modis_files=ListofExtensionAndName(download_path,'hdf')
            modis_status=True
        ###################################################
        else:
            modis_next_file=ListofExtensionAndName(folder_list[index],'hdf')
            if len(modis_next_file[0])>1:                
                #print(download_path)
                modis_files=ListofExtensionAndName(download_path,'hdf')
                modis_status=True
            #every 5 minutes. check the target file
            else:            
                modis_status=False
                time.sleep(300)
    
    
    try:
        #create tif file under target file > data/Modis_Fire_Date/2001/2001.01.01/tif
        os.mkdir(download_path+'tif')
    except:
        pass
    
    for index2,img in enumerate(modis_files[0]):
        convert_image=False
        counter=0
        while convert_image is False:
            try:
                # we get this information from gdalinfo output.
                # gdalinfo target_file.hdf
                img_path='HDF4_EOS:EOS_GRID:'+'"{}"'.format(img)+':MOD_Grid_Monthly_500m_DB_BA:"Burn Date"'
                #print(img_path)
                hdf_ds = gdal.Open(img_path, gdal.GA_ReadOnly)
                array_img=hdf_ds.ReadAsArray()
                #np_where_img = numpy.where((array_img<0),-32768,(array_img))
                s_srs = hdf_ds.GetProjectionRef()
                osng = osr.SpatialReference ()
                osng.SetFromUserInput (s_srs)
                geo_t = hdf_ds.GetGeoTransform ()
                x_size = hdf_ds.RasterXSize # Raster xsize
                y_size = hdf_ds.RasterYSize # Raster ysize
                #create gdal memory file
                mem_drv = gdal.GetDriverByName( 'MEM' )
                dest = mem_drv.Create('', x_size,y_size, 1, gdal.GDT_Int16)
                dest.SetGeoTransform( geo_t )
                dest.SetProjection ( osng.ExportToWkt() )
                #dest.GetRasterBand(1).SetNoDataValue(-32768)
                dest.GetRasterBand(1).WriteArray(array_img)
                #dest.GetRasterBand(1).WriteArray(np_where_img)
                
                output_path=download_path+'tif/'+modis_files[1][index2]+'.tif'
                #print(output_path)
                gdal.Warp(output_path, dest, format = 'GTiff')
                #gdal.Warp(output_path, dest, format = 'GTiff',srcNodata=-32768,dstNodata=-32768)
                dst_ds=None
                mem_drv=None
                convert_image=True
            except:
                convert_image=False
                counter+=1
                
                if counter==10:
                    with open(download_path+'error_list', 'a') as t:
                        t.write("%s\n" % img)                
                    print(img + ' couldnt be converted')
                    print("Unexpected error:", sys.exc_info()[0])
                    break
                continue